In [1]:
#model套件
import torch
from torch import nn as nn
from torch.nn.parameter import Parameter as Parameter
from torch.utils.data import Dataset
from torch.autograd import Variable as Variable
from torchsummary import summary
import torch.nn.functional as F

#聲音處理套件
import librosa
#資料處理套件
import pandas as pd
import numpy as np
#時間套件
import time

from enum import Enum

In [2]:
#存檔位置
ModelMode = "ResNet_ESC50_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
filePath_Train = ModelMode + TrainDataSoundMode[4] + "_EpochUp_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[4] +  "_EpochUp_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[4] + "_EpochUp"

In [3]:
#音訊資訊CSV檔位置
DATA_CSV_PATH = "D:/Hu_code/ESC-50-master/ESC-50-master/meta/esc50.csv"
#音訊存放位置
DATA_PATH = "D:/Hu_code/ESC-50-master/ESC-50-master/audio/"

def loadDataESC50(data_path, data_csv_path, soundLength):
    dataX_Fold = []
    dataY_Fold = []
    
    #讀取CSV檔
    df = pd.read_csv(data_csv_path)

    #計數
    count = 0
    
    for Fold in range(1, 6, 1):
        #初始化
        dataX = []
        dataY = []
        #篩選資料
        sound_df = df[df.fold == Fold]
        #轉換train資料/添加標籤
        for data in sound_df.itertuples():
            sig, sr = librosa.load(data_path+data[1])
            for i in range((6-soundLength)):
                sound = sig[i*sr : int((i+soundLength)*sr)]
                sound = audio_to_melspectrogram(audio=sound, sr=sr)
                dataX.append(sound)
                dataY.append(data[3])       
            count +=1
            print("{:.3f}".format(100 * count / len(df))+"%", end='\r')
        dataX_Fold.append(dataX)
        dataY_Fold.append(dataY)
                
    #通通轉成 torch 的 tensor 形式
    dataX_Fold = torch.Tensor(dataX_Fold)
    dataY_Fold = torch.Tensor(dataY_Fold).type(torch.LongTensor)
    
    #轉維度 FBCHW
    dataX_Fold = torch.reshape(dataX_Fold, (dataX_Fold.shape[0], dataX_Fold.shape[1], 1, dataX_Fold.shape[2], dataX_Fold.shape[3]))
           
    # 完成時打印最終消息
    print("Progress: 100% - Completed")
    return dataX_Fold, dataY_Fold

class conf:
    sr = 44100  #採樣率
    hop_length = 512  #帧移
    fmin = 20  #最低頻率
    fmax = sr // 2  #最高頻率
    n_mels = 128  #濾波器個數
    n_fft = 2048  #FFT窗口長度

#轉頻譜圖
def audio_to_melspectrogram(audio, sr):
    spectrogram = librosa.feature.melspectrogram(y=audio, 
                                                 sr=sr,
                                                 n_mels=128,
                                                 hop_length=512,
                                                 n_fft=2048,
                                                 fmin=20,
                                                 fmax=sr//2)
    spectrogram = librosa.power_to_db(spectrogram)
    return spectrogram

class CustomDatasetESC50(Dataset):
    def __init__(self, inputDataX, inputDataY, fold, isTrain):
        #防止範圍錯誤
        if(fold>5 or fold <1):
            raise ValueError("Fold範圍錯誤")
            
        #資料初始化
        self.dataX = []
        self.dataY = []
        
        # True:訓練資料  False:測試資料
        if isTrain:
            for Fold in range(1, 6, 1):
                if(Fold != fold):
                    for X in inputDataX[Fold-1]:
                        self.dataX.append(X)
                    for Y in inputDataY[Fold-1]:
                        self.dataY.append(Y)
        else:
            self.dataX = inputDataX[fold-1]
            self.dataY = inputDataY[fold-1]

    def __len__(self):
        return len(self.dataX)
    
    def __getitem__(self, index):
        # 根據索引返回單個樣本及其標籤
        data = self.dataX[index]
        label = self.dataY[index]
        
        return data, label

In [4]:
#殘差網路模塊
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, downsample):
        super().__init__()
        if downsample:
            self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1)
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=2),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
            self.shortcut = nn.Sequential()

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, input):
        shortcut = self.shortcut(input)
        input = nn.ReLU()(self.bn1(self.conv1(input)))
        input = nn.ReLU()(self.bn2(self.conv2(input)))
        input = input + shortcut
        return nn.ReLU()(input)
    
#ResNet_18+注意力機制 模型
class ResNet_18(nn.Module):
    def __init__(self, nums_class=10):
        super(ResNet_18, self).__init__()
        
        self.layer_0 = nn.Sequential(
            # width_of_output = (width_of_input - filter_size + 2*padding)/stride + 1
            nn.Conv2d(in_channels = 1, out_channels = 64, kernel_size = (7,7), stride = (2,2), padding = 3),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        
        #殘差層
        #-----------------------------------------------------
        self.layer_1 = nn.Sequential(
            ResBlock(64, 128, downsample=True),
            ResBlock(128, 128, downsample=False)
        )
        
        self.layer_2 = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(128, 64),
            nn.Linear(64, nums_class)
        )
        
    def forward(self, x):
        x = self.layer_0(x)
        x = self.layer_1(x)
        x = self.layer_2(x)
        return x



In [5]:
#訓練模型(全部訓練)
def train_model_TrainAll(trainFold, setLR, setEpoch):
    #用來記錄全部訓練過程的準確度和loss
    train_loss_array = []
    train_accuracy_array = []
    
    #---------------------------------------------------
    #訓練相關參數
    #---------------------------------------------------
    #訓練次數
    num_epochs = setEpoch
    #批次大小
    BATCH_SIZE = 64
    #學習率
    LR = setLR
    #loss方法
    loss_func = nn.CrossEntropyLoss()
    #優化器
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #scheduler
    #--------------------------------------------------------------------
    #調度器
    scheduler  = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #資料抓入
    #預處理
    #--------------------------------------------------------------------
    #資料抓入
    trainloader = CustomDatasetESC50(inputDataX = dataX,
                               inputDataY = dataY,
                               fold = trainFold,
                               isTrain = True)
    #分批方法
    train_loader = torch.utils.data.DataLoader(trainloader, batch_size = BATCH_SIZE, shuffle = True)
    #--------------------------------------------------------------------
    
    #訓練次數
    for epoch in range(num_epochs):
        #訓練準確度
        correct_train = 0
        #訓練資料總筆數
        total_train = 0
        #批次訓練
        for i, data in enumerate(train_loader):
            #開啟模型訓練
            model.train()
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            #清除梯度
            optimizer.zero_grad()
            #向前傳播
            outputs = model(inputs)
            #計算loss
            train_loss = loss_func(outputs, labels)
            #反向傳播
            train_loss.backward()
            #梯度下降
            optimizer.step()
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_train += len(labels)
            #計算判斷正確的筆數
            correct_train += (predicted == labels).float().sum()
        #計算這次訓練的準確度
        train_accuracy = 100 * correct_train / float(total_train)
        #紀錄這次訓練的準確度
        train_accuracy_array.append(train_accuracy)
        #紀錄這次訓練的loss
        train_loss_array.append(train_loss.data)
        
        #印出本次訓練結果
        print('Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}%'.format(epoch+1, num_epochs, train_loss.data, train_accuracy))
        #調度器走一步
        scheduler.step()
        
    #測試準確度
    test_accuracy1 = test_model(inputDataX = dataX1, inputDataY = dataY1, testFold = trainFold)
    test_accuracy3 = test_model(inputDataX = dataX3, inputDataY = dataY3, testFold = trainFold)
    test_accuracy5 = test_model(inputDataX = dataX5, inputDataY = dataY5, testFold = trainFold)
        
    #將gpu資料取回
    train_loss_array = torch.stack(train_loss_array,dim=-1).cpu()
    train_accuracy_array = torch.stack(train_accuracy_array,dim=-1).cpu()
    
    return train_loss_array, train_accuracy_array, test_accuracy1, test_accuracy3, test_accuracy5

In [6]:
#測試模型的準確度
def test_model(inputDataX, inputDataY, testFold):
    #固定模型不改變權重
    model.eval()
    #計數初始化
    correct = 0
    total = 0
    #資料抓取
    testloader = CustomDatasetESC50(inputDataX = inputDataX,
                               inputDataY = inputDataY,
                               fold = testFold,
                               isTrain = False)
    #封裝                                
    test_loader = torch.utils.data.DataLoader(testloader, batch_size=64, shuffle=False)
    
    #不計算梯度 加速計算
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            
            #預測結果
            outputs = model(inputs)
            
            #獲取預測標籤
            _, predicted = torch.max(outputs.data, 1)
            
            #計算準確度
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Test accuracy of the model: {:.6f}%'.format(100 * correct / total))
    return (100 * correct / total)

In [7]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
dataX1, dataY1 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 1)
print("讀取3秒資料")
dataX3, dataY3 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 3)
print("讀取5秒資料")
dataX5, dataY5 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料


C:\Users\test\AppData\Local\Temp\ipykernel_1244\2625783892.py:36: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  dataX_Fold = torch.Tensor(dataX_Fold)


Progress: 100% - Completed
讀取3秒資料
Progress: 100% - Completed
讀取5秒資料
Progress: 100% - Completed
Data Loading Time: 133.96 seconds


In [ ]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []

#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    kTLTrainTime = []
    kTLTrainAccuracy = []
    kTLTestAccuracy1 = []
    kTLTestAccuracy3 = []
    kTLTestAccuracy5 = []

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = ResNet_18(nums_class = 50)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    
    #學習率
    lr=0.0032
    #epoch
    epoch=0
    for TL in range(3):
        print("----------------------------------------------------------------------------------------------------------------")
        print("fold: ", fold, "TL: ", TL)
        print("----------------------------------------------------------------------------------------------------------------")
        #訓練資料(要用幾秒訓練)
        if(TL==0):
            #1s
            dataX = dataX5
            dataY = dataY5
            epoch = 20
        elif(TL==1):
            #3s
            dataX = dataX3
            dataY = dataY3
            epoch = 40
        else:
            #5s
            dataX = dataX1
            dataY = dataY1
            epoch = 60

        #紀錄訓練開始時間
        start_time = time.time()

        #訓練模型(只訓練分類器)
        train_loss, train_accuracy, test_accuracy1, test_accuracy3, test_accuracy5 = train_model_TrainAll(trainFold=fold, setLR=lr, setEpoch=epoch)

        #紀錄訓練結束時間
        end_time = time.time()

        #計算訓練耗時
        training_time = end_time - start_time

        #印出訓練耗時
        print("Training Time: {:.2f} seconds".format(training_time))

        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("訓練過程\n")
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        for i in range(len(train_loss)):
            # 寫入資料
            f.write("Fold: {} TL: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, TL, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
        f.write("Training Time: {:.2f} seconds\n".format(training_time))
        f.write("---------------------------------------------------------------------------------------------------------------\n")

        #--------------------------------------------------------------------
        
        kTLTrainTime.append(training_time)
        kTLTrainAccuracy.append(train_accuracy[-1])
        kTLTestAccuracy1.append(test_accuracy1)
        kTLTestAccuracy3.append(test_accuracy3)
        kTLTestAccuracy5.append(test_accuracy5)
        
    kFoldTrainTime.append(kTLTrainTime)
    kFoldTrainAccuracy.append(kTLTrainAccuracy)
    kFoldTestAccuracy1.append(kTLTestAccuracy1)
    kFoldTestAccuracy3.append(kTLTestAccuracy3)
    kFoldTestAccuracy5.append(kTLTestAccuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

可用的 GPU 数量: 1
----------------------------------------------------------------------------------------------------------------
fold:  1 TL:  0
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/20 Train_Loss: 3.2500662803649902 Train_acc: 9.812500%
Train Epoch: 2/20 Train_Loss: 2.375426769256592 Train_acc: 19.625000%
Train Epoch: 3/20 Train_Loss: 2.42807936668396 Train_acc: 28.500000%
Train Epoch: 4/20 Train_Loss: 2.47050404548645 Train_acc: 26.937500%
Train Epoch: 5/20 Train_Loss: 1.9884859323501587 Train_acc: 33.125000%
Train Epoch: 6/20 Train_Loss: 1.999708890914917 Train_acc: 38.062500%
Train Epoch: 7/20 Train_Loss: 1.6675856113433838 Train_acc: 40.062500%
Train Epoch: 8/20 Train_Loss: 1.9186574220657349 Train_acc: 45.000000%
Train Epoch: 9/20 Train_Loss: 1.6097002029418945 Train_acc: 47.000000%
Train Epoch: 10/20 Train_Loss: 1.436065912246704 Train_acc: 47.000000%
Train Epoch: 11/20 Train_Loss: 1.73791372

Train Epoch: 41/60 Train_Loss: 0.28981757164001465 Train_acc: 92.350006%
Train Epoch: 42/60 Train_Loss: 0.28019556403160095 Train_acc: 92.450005%
Train Epoch: 43/60 Train_Loss: 0.3584262728691101 Train_acc: 92.425003%
Train Epoch: 44/60 Train_Loss: 0.22641025483608246 Train_acc: 92.462502%
Train Epoch: 45/60 Train_Loss: 0.38427603244781494 Train_acc: 92.425003%
Train Epoch: 46/60 Train_Loss: 0.1945486217737198 Train_acc: 92.737503%
Train Epoch: 47/60 Train_Loss: 0.14161911606788635 Train_acc: 92.900002%
Train Epoch: 48/60 Train_Loss: 0.2002555876970291 Train_acc: 92.862503%
Train Epoch: 49/60 Train_Loss: 0.1568014770746231 Train_acc: 92.912506%
Train Epoch: 50/60 Train_Loss: 0.1956942230463028 Train_acc: 93.075005%
Train Epoch: 51/60 Train_Loss: 0.07103154808282852 Train_acc: 92.950005%
Train Epoch: 52/60 Train_Loss: 0.22500714659690857 Train_acc: 92.950005%
Train Epoch: 53/60 Train_Loss: 0.13823309540748596 Train_acc: 92.862503%
Train Epoch: 54/60 Train_Loss: 0.2373846471309662 Train_

Train Epoch: 19/60 Train_Loss: 0.4169490933418274 Train_acc: 90.725006%
Train Epoch: 20/60 Train_Loss: 0.2698409855365753 Train_acc: 91.012505%
Train Epoch: 21/60 Train_Loss: 0.2060767263174057 Train_acc: 91.150002%
Train Epoch: 22/60 Train_Loss: 0.21733741462230682 Train_acc: 91.300003%
Train Epoch: 23/60 Train_Loss: 0.5082805752754211 Train_acc: 91.000008%
Train Epoch: 24/60 Train_Loss: 0.384725421667099 Train_acc: 90.825005%
Train Epoch: 25/60 Train_Loss: 0.24259822070598602 Train_acc: 91.000008%
Train Epoch: 26/60 Train_Loss: 0.09956885129213333 Train_acc: 90.850006%
Train Epoch: 27/60 Train_Loss: 0.37863439321517944 Train_acc: 90.325005%
Train Epoch: 28/60 Train_Loss: 0.32637155055999756 Train_acc: 89.537506%
Train Epoch: 29/60 Train_Loss: 0.3058609962463379 Train_acc: 89.737503%
Train Epoch: 30/60 Train_Loss: 0.20257800817489624 Train_acc: 90.487503%
Train Epoch: 31/60 Train_Loss: 0.09261885285377502 Train_acc: 91.975006%
Train Epoch: 32/60 Train_Loss: 0.2036065012216568 Train_ac

Train Epoch: 1/60 Train_Loss: 1.123884677886963 Train_acc: 68.387505%
Train Epoch: 2/60 Train_Loss: 0.8749858736991882 Train_acc: 74.250000%
Train Epoch: 3/60 Train_Loss: 0.6952130794525146 Train_acc: 74.625000%
Train Epoch: 4/60 Train_Loss: 0.9085500836372375 Train_acc: 77.237503%
Train Epoch: 5/60 Train_Loss: 0.6573989987373352 Train_acc: 78.787506%
Train Epoch: 6/60 Train_Loss: 0.6033310890197754 Train_acc: 79.425003%
Train Epoch: 7/60 Train_Loss: 0.5963903069496155 Train_acc: 80.762505%
Train Epoch: 8/60 Train_Loss: 0.5321685075759888 Train_acc: 81.187508%
Train Epoch: 9/60 Train_Loss: 0.5656734704971313 Train_acc: 82.475006%
Train Epoch: 10/60 Train_Loss: 0.6955076456069946 Train_acc: 83.262505%
Train Epoch: 11/60 Train_Loss: 0.5850108861923218 Train_acc: 84.212502%
Train Epoch: 12/60 Train_Loss: 0.39371225237846375 Train_acc: 84.612503%
Train Epoch: 13/60 Train_Loss: 0.45575881004333496 Train_acc: 83.837502%
Train Epoch: 14/60 Train_Loss: 0.49972161650657654 Train_acc: 84.050003%

Train Epoch: 24/40 Train_Loss: 0.21445928514003754 Train_acc: 93.312500%
Train Epoch: 25/40 Train_Loss: 0.14217576384544373 Train_acc: 93.333336%
Train Epoch: 26/40 Train_Loss: 0.12726910412311554 Train_acc: 93.375000%
Train Epoch: 27/40 Train_Loss: 0.15835529565811157 Train_acc: 92.770836%
Train Epoch: 28/40 Train_Loss: 0.36261725425720215 Train_acc: 92.687500%
Train Epoch: 29/40 Train_Loss: 0.30938953161239624 Train_acc: 93.645836%
Train Epoch: 30/40 Train_Loss: 0.30723822116851807 Train_acc: 94.687500%
Train Epoch: 31/40 Train_Loss: 0.09834478050470352 Train_acc: 95.437500%
Train Epoch: 32/40 Train_Loss: 0.08757958561182022 Train_acc: 95.937500%
Train Epoch: 33/40 Train_Loss: 0.18261674046516418 Train_acc: 95.979172%
Train Epoch: 34/40 Train_Loss: 0.30283915996551514 Train_acc: 96.062500%
Train Epoch: 35/40 Train_Loss: 0.08907212316989899 Train_acc: 96.166672%
Train Epoch: 36/40 Train_Loss: 0.11370109021663666 Train_acc: 96.291672%
Train Epoch: 37/40 Train_Loss: 0.13525794446468353 

Train Epoch: 2/40 Train_Loss: 1.2564512491226196 Train_acc: 61.520836%
Train Epoch: 3/40 Train_Loss: 1.0955435037612915 Train_acc: 63.729168%
Train Epoch: 4/40 Train_Loss: 1.099627137184143 Train_acc: 64.979172%
Train Epoch: 5/40 Train_Loss: 0.8089838027954102 Train_acc: 71.187500%
Train Epoch: 6/40 Train_Loss: 1.0373873710632324 Train_acc: 72.750000%
Train Epoch: 7/40 Train_Loss: 0.8946148753166199 Train_acc: 72.541672%
Train Epoch: 8/40 Train_Loss: 0.9856854677200317 Train_acc: 75.125000%
Train Epoch: 9/40 Train_Loss: 0.8500512838363647 Train_acc: 76.562500%
Train Epoch: 10/40 Train_Loss: 0.8162871599197388 Train_acc: 78.666672%
Train Epoch: 11/40 Train_Loss: 0.4026501178741455 Train_acc: 82.375000%
Train Epoch: 12/40 Train_Loss: 0.41531291604042053 Train_acc: 81.354172%
Train Epoch: 13/40 Train_Loss: 0.7791510224342346 Train_acc: 83.958336%
Train Epoch: 14/40 Train_Loss: 0.5544447898864746 Train_acc: 82.625000%
Train Epoch: 15/40 Train_Loss: 0.5750924348831177 Train_acc: 85.166672%


In [ ]:
class FilterMode(Enum):
    Max = "Max"
    Min = "Min"
    Mean = "Mean"
    
def listFilter(data, mode:FilterMode):
    TL = []
    for fold in range(len(data)):
        TL.append(data[fold][len(data[fold])-1])
    
    switch = {
        FilterMode.Max: max(TL),
        FilterMode.Min: min(TL),
        FilterMode.Mean: np.mean(TL)
    }

    return switch.get(mode, -1)

#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    for TL in range(3):
        # 寫入資料
        f.write("Fold: {} TL: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, TL, kFoldTrainAccuracy[fold][TL], kFoldTestAccuracy1[fold][TL], kFoldTestAccuracy3[fold][TL], kFoldTestAccuracy5[fold][TL], kFoldTrainTime[fold][TL]))
    f.write("-----------------------------------------------------------------------------------\n")
    
# 寫入資料[]
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Max), listFilter(kFoldTestAccuracy1, FilterMode.Max), listFilter(kFoldTestAccuracy3, FilterMode.Max), listFilter(kFoldTestAccuracy5, FilterMode.Max)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Min), listFilter(kFoldTestAccuracy1, FilterMode.Min), listFilter(kFoldTestAccuracy3, FilterMode.Min), listFilter(kFoldTestAccuracy5, FilterMode.Min)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Mean), listFilter(kFoldTestAccuracy1, FilterMode.Mean), listFilter(kFoldTestAccuracy3, FilterMode.Mean), listFilter(kFoldTestAccuracy5, FilterMode.Mean)))

f.close()